## Import dataset
#### basic metrics calculation

In [2]:
# imports

import pandas as pd
import numpy as np
import networkx as nx
from random import sample
import time

In [3]:
# load dataset (reddit social graph)
dataset = 'HU_edges.csv' #'large_twitch_edges.csv'

data = pd.read_csv(dataset, header = 0, sep = ',') 
data = data[[data.columns[0], data.columns[1]]]
data.head()

,node_1,node_2
0,0,24208
1,0,24445
2,0,18055
3,0,26575
4,0,12596


In [4]:
# Make networkx graph object

graph = nx.from_pandas_edgelist(data, data.columns[0], data.columns[1])
#graph_directed = nx.from_pandas_edgelist(data, data.columns[0], data.columns[1], create_using = nx.DiGraph)

In [5]:
# graph properties

print('Number of nodes: ', graph.number_of_nodes())
print('Number of edges: ', graph.number_of_edges())

Number of nodes:  47538
Number of edges:  222887


In [6]:
# Giant component
Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
print('Number of largest connected components: ', len(Gcc))

giant = graph.subgraph(Gcc[0])
print('Number of nodes in giant component: ', giant.number_of_nodes())

Number of largest connected components:  1
Number of nodes in giant component:  47538


In [6]:
data_undi = data.drop_duplicates()

In [70]:
print('Average Clustering coefficient: ', nx.average_clustering(graph))

Average Clustering coefficient:  0.18090101843790804


In [7]:
# input preparation for Rigel
np.random.seed(1337)
nodes = list(giant.nodes())
N = len(nodes)
landmark_cnt = 100
landmarks = sample(nodes, landmark_cnt)  #here a landmark selection strategy could be introduced
landmark_indices = [nodes.index(lm) for lm in landmarks]
distance_matrix = {j:dict() for j in landmarks}
#distance_matrix = dict()
start_time = time.time()
for i in landmarks:
    #distance_matrix[i] = dict()
    #print('landmark ', i, ' calculating...')
    path_lengths = nx.single_source_shortest_path_length(giant, i)
    for j in nodes:
        distance_matrix[i][j] = int(path_lengths[j])
    print("landmark ", i," finished in %s seconds " % (time.time() - start_time))

landmark  43741  finished in 0.48421454429626465 seconds 
landmark  23411  finished in 1.063673973083496 seconds 
landmark  37280  finished in 1.6541101932525635 seconds 
landmark  29027  finished in 2.2139697074890137 seconds 
landmark  11934  finished in 2.779066801071167 seconds 
landmark  41821  finished in 3.380887031555176 seconds 
landmark  24882  finished in 3.926957130432129 seconds 
landmark  9825  finished in 4.444339990615845 seconds 
landmark  30631  finished in 4.962022066116333 seconds 
landmark  17220  finished in 5.494752407073975 seconds 
landmark  24741  finished in 6.028928756713867 seconds 
landmark  2095  finished in 6.5310869216918945 seconds 
landmark  37678  finished in 7.078580379486084 seconds 
landmark  22618  finished in 7.610554456710815 seconds 
landmark  33000  finished in 8.111961364746094 seconds 
landmark  6021  finished in 8.682451963424683 seconds 
landmark  4490  finished in 9.340370893478394 seconds 
landmark  13622  finished in 9.952441215515137 

In [8]:
with open('l_deezer.txt', 'w') as f:
    for i in distance_matrix.keys():
        nested_list = [str(j) for j in list(distance_matrix[i].values())]
        f.writelines('\t'.join(nested_list) + '\n')
f.close()

In [9]:
with open('r_deezer.txt', 'w') as f:
    for node in landmark_indices:
        f.write('%s\n' % node)
f.close()

In [10]:
with open('0_deezer.ord', 'w') as f:
    for i in range(N):
        if not i in landmark_indices:
            f.write('%s\n' % i)
f.close()

In [11]:
len(nodes)

47538